# 📐 Gradientenabstieg am Mini-Beispiel - Vorlesung 07

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_07/gradient_descent_mini_example.ipynb)

## 🎯 Das Energie-Temperatur-Beispiel aus der Vorlesung

Wir haben drei Messpunkte vom **Energieverbrauch einer Kühlanlage** bei verschiedenen **Außentemperaturen**:
- Punkt 1: (10°C, 14 kWh/h) 
- Punkt 2: (20°C, 19 kWh/h)
- Punkt 3: (30°C, 25 kWh/h)

**Ziel**: Finde die beste Gerade $y = mx + b$ durch diese Punkte!

**Warum macht das Sinn?** 🤔
- ❄️ **Niedrigere Temperaturen** → Weniger Kühlbedarf → Geringerer Energieverbrauch  
- ☀️ **Höhere Temperaturen** → Mehr Kühlbedarf → Höherer Energieverbrauch
- 📈 **Vorhersagen möglich**: Was passiert bei 0°C? Bei 40°C? Bei 25°C?

**In der Vorlesung haben wir gelernt:**
- ✅ **Normalgleichungen** - Die exakte analytische Lösung
- ✅ **Matrixform** - $\boldsymbol{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$
- ✅ **MSE minimieren** - Das Least-Squares-Prinzip

**Aber was passiert, wenn Normalgleichungen nicht funktionieren?**
- 🔴 **Nichtlineare Modelle** (z.B. $y = ae^{bx}$)
- 🔴 **Millionen von Parametern** (neuronale Netze)
- 🔴 **Große Datensätze** (Matrix-Inversion zu langsam)

**Antwort: Gradientenabstieg!** 🚀

Wir schauen uns jetzt den Gradientenabstieg an diesem einfachen Beispiel an!

## 🗺️ Was lernen wir heute?

✅ **MSE-Landschaft visualisieren** - Wie sieht das "Gebirge" aus?  
✅ **Gradientenabstieg Schritt für Schritt** - Wie findet der Computer den Weg bergab?  
✅ **Parameter-Optimierung verstehen** - Warum funktioniert das Verfahren?  
✅ **Verbindung zu neuronalen Netzen** - Das gleiche Prinzip bei Millionen Parametern!  
✅ **Wann Normalgleichungen vs. Gradientenabstieg** - Die richtige Wahl treffen

## 🔧 Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# Für schöne Plots
%matplotlib inline
# Versuche verschiedene Styles (fallback wenn seaborn nicht verfügbar)
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')
        
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("🎉 Alle Libraries geladen!")
print("📊 Bereit für Gradientenabstieg-Visualisierung!")

## 📊 Unsere Datenpunkte - Das Kühlanlage-Temperatur-Beispiel

In [ ]:
# 🎯 STARTPUNKT-KONFIGURATION für Gradientenabstieg
# =================================================================
# HIER KANNST DU VERSCHIEDENE STARTPUNKTE AUSPROBIEREN!
# Diese Variablen werden in allen nachfolgenden Visualisierungen verwendet

# Startpunkt 1: Bewusst schlecht gewählt (Standard)
#START_M = 0.3   # Steigung
#START_B = 16.0  # Y-Achsenabschnitt
START_M = 0   # Steigung
START_B = 0  # Y-Achsenabschnitt

# Alternative Startpunkte zum Testen (einfach auskommentieren):
# START_M, START_B = 0.1, 20.0   # Sehr flache Steigung, hoher Y-Achsenabschnitt
# START_M, START_B = 0.8, 5.0    # Steile Steigung, niedriger Y-Achsenabschnitt  
# START_M, START_B = 1.0, 0.0    # Sehr steile Steigung, kein Y-Achsenabschnitt
# START_M, START_B = 0.0, 15.0   # Horizontale Linie
# START_M, START_B = -0.2, 25.0  # Negative Steigung (unphysikalisch!)

print("=== STARTPUNKT-KONFIGURATION ===")
print(f"Gewählter Startpunkt: m = {START_M}, b = {START_B}")

# Kompatibilität: Setze auch die alten Variablen
m_guess, b_guess = START_M, START_B

print(">>> Startpunkt für alle Visualisierungen und Gradientenabstieg gesetzt!")
print(">>> Du kannst verschiedene Startpunkte ausprobieren, indem du die Werte oben änderst.")

In [ ]:
# Die Datenpunkte aus der Vorlesung (exakt wie im Skript)
x_data = np.array([10, 20, 30])  # Außentemperatur in °C
y_data = np.array([14, 19, 25])  # Energieverbrauch in kWh/h

print("📋 Unsere Datenpunkte (exakt aus Vorlesung 07):")
print("Temperatur (°C) | Energieverbrauch (kWh/h)")
print("----------------|------------------------")
for i in range(len(x_data)):
    print(f"      {x_data[i]:2d}        |          {y_data[i]:2d}")

# Analytische Lösung berechnen (wie im Skript)
n = len(x_data)
sum_x = np.sum(x_data)      # Σx_i = 60
sum_y = np.sum(y_data)      # Σy_i = 58  
sum_xy = np.sum(x_data * y_data)  # Σx_i*y_i berechnen
sum_x2 = np.sum(x_data**2)  # Σx_i² = 1400

print(f"\n🧮 Berechnungen für Normalgleichungen (wie im Skript):")
print(f"   n = {n}")
print(f"   Σx_i = {sum_x}")
print(f"   Σy_i = {sum_y}")
print(f"   Σx_i*y_i = {sum_xy}")
print(f"   Σx_i² = {sum_x2}")

# Normalgleichungen anwenden
m_analytical = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
b_analytical = (sum_y - m_analytical * sum_x) / n

print(f"\n🎯 Analytische Lösung (Normalgleichungen):")
print(f"   m = ({n}×{sum_xy} - {sum_x}×{sum_y}) / ({n}×{sum_x2} - {sum_x}²)")
print(f"   m = ({n*sum_xy} - {sum_x*sum_y}) / ({n*sum_x2} - {sum_x**2})")
print(f"   m = {n*sum_xy - sum_x*sum_y} / {n*sum_x2 - sum_x**2} = {m_analytical:.3f}")
print(f"   b = ({sum_y} - {m_analytical:.3f}×{sum_x}) / {n} = {b_analytical:.3f}")

mse_analytical = np.mean((y_data - (m_analytical * x_data + b_analytical))**2)
print(f"\n📊 Optimaler MSE: {mse_analytical:.6f}")

# 🔧 KONTROLLE: Berechnung mit Library-Funktionen
print(f"\n🔧 KONTROLLE mit Standard-Libraries:")

# Methode 1: numpy.polyfit (Polynom-Fitting, Grad 1 = Gerade)
polyfit_coeff = np.polyfit(x_data, y_data, deg=1)
m_polyfit, b_polyfit = polyfit_coeff[0], polyfit_coeff[1]
mse_polyfit = np.mean((y_data - (m_polyfit * x_data + b_polyfit))**2)

print(f"\n1️⃣ np.polyfit(deg=1):")
print(f"   m = {m_polyfit:.6f}")
print(f"   b = {b_polyfit:.6f}")
print(f"   MSE = {mse_polyfit:.8f}")

# Methode 2: Least Squares mit numpy.linalg.lstsq
X_matrix = np.column_stack([x_data, np.ones(len(x_data))])  # [x, 1] Matrix
lstsq_result = np.linalg.lstsq(X_matrix, y_data, rcond=None)
m_lstsq, b_lstsq = lstsq_result[0][0], lstsq_result[0][1]
mse_lstsq = np.mean((y_data - (m_lstsq * x_data + b_lstsq))**2)

print(f"\n2️⃣ np.linalg.lstsq (Matrix-Lösung):")
print(f"   m = {m_lstsq:.6f}")
print(f"   b = {b_lstsq:.6f}")
print(f"   MSE = {mse_lstsq:.8f}")

# Methode 3: scipy.stats.linregress (falls verfügbar)
try:
    from scipy.stats import linregress
    linreg_result = linregress(x_data, y_data)
    m_scipy, b_scipy = linreg_result.slope, linreg_result.intercept
    mse_scipy = np.mean((y_data - (m_scipy * x_data + b_scipy))**2)
    
    print(f"\n3️⃣ scipy.stats.linregress:")
    print(f"   m = {m_scipy:.6f}")
    print(f"   b = {b_scipy:.6f}")
    print(f"   MSE = {mse_scipy:.8f}")
    print(f"   R² = {linreg_result.rvalue**2:.6f}")
    print(f"   p-value = {linreg_result.pvalue:.8f}")
except ImportError:
    print(f"\n3️⃣ scipy.stats.linregress: Nicht verfügbar")
    m_scipy, b_scipy = m_analytical, b_analytical  # Fallback

# Vergleichstabelle
print(f"\n📊 VERGLEICHSTABELLE:")
print(f"{'Methode':<20} {'m (Steigung)':<15} {'b (Achsenabschnitt)':<20} {'MSE':<15}")
print(f"{'-'*20} {'-'*15} {'-'*20} {'-'*15}")
print(f"{'Normalgleichungen':<20} {m_analytical:<15.6f} {b_analytical:<20.6f} {mse_analytical:<15.8f}")
print(f"{'np.polyfit':<20} {m_polyfit:<15.6f} {b_polyfit:<20.6f} {mse_polyfit:<15.8f}")
print(f"{'np.linalg.lstsq':<20} {m_lstsq:<15.6f} {b_lstsq:<20.6f} {mse_lstsq:<15.8f}")
try:
    print(f"{'scipy.linregress':<20} {m_scipy:<15.6f} {b_scipy:<20.6f} {mse_scipy:<15.8f}")
except:
    pass

# Differenzen berechnen
diff_m_polyfit = abs(m_analytical - m_polyfit)
diff_b_polyfit = abs(b_analytical - b_polyfit)
diff_m_lstsq = abs(m_analytical - m_lstsq)
diff_b_lstsq = abs(b_analytical - b_lstsq)

print(f"\n✅ ÜBEREINSTIMMUNG:")
print(f"   np.polyfit:     Δm = {diff_m_polyfit:.10f}, Δb = {diff_b_polyfit:.10f}")
print(f"   np.linalg.lstsq: Δm = {diff_m_lstsq:.10f}, Δb = {diff_b_lstsq:.10f}")

if diff_m_polyfit < 1e-10 and diff_b_polyfit < 1e-10:
    print(f"🎯 Alle Methoden liefern IDENTISCHE Ergebnisse!")
    print(f"💡 Das bestätigt unsere Normalgleichungen-Implementierung!")
else:
    print(f"⚠️  Kleine numerische Unterschiede (normal bei Floating-Point)")

# Plotten der Datenpunkte mit verschiedenen Lösungen
plt.figure(figsize=(12, 8))
plt.scatter(x_data, y_data, color='red', s=150, zorder=5, edgecolors='black', linewidth=2)
plt.xlabel('Außentemperatur (°C)', fontweight='bold')
plt.ylabel('Energieverbrauch Kühlung (kWh/h)', fontweight='bold')
plt.title('Kühlanlage: Vergleich verschiedener Berechnungsmethoden', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xlim(0, 40)
plt.ylim(5, 35)

# Alle Lösungen einzeichnen (sollten identisch sein)
x_line = np.linspace(0, 40, 100)
y_analytical = m_analytical * x_line + b_analytical
y_polyfit = m_polyfit * x_line + b_polyfit
y_lstsq = m_lstsq * x_line + b_lstsq

plt.plot(x_line, y_analytical, 'b-', linewidth=3, alpha=0.8, 
         label=f'Normalgleichungen: y = {m_analytical:.3f}x + {b_analytical:.2f}')
plt.plot(x_line, y_polyfit, 'g--', linewidth=2, alpha=0.7,
         label=f'np.polyfit: y = {m_polyfit:.3f}x + {b_polyfit:.2f}')
plt.plot(x_line, y_lstsq, 'r:', linewidth=2, alpha=0.7,
         label=f'np.linalg.lstsq: y = {m_lstsq:.3f}x + {b_lstsq:.2f}')

# Punkte beschriften
for i, (x, y) in enumerate(zip(x_data, y_data)):
    plt.annotate(f'({x}°C, {y} kWh/h)', (x, y), xytext=(5, 5), textcoords='offset points', 
                fontweight='bold', fontsize=10, bbox=dict(boxstyle='round,pad=0.2', facecolor='yellow', alpha=0.7))

plt.legend()
plt.tight_layout()
plt.show()

print(f"\n🎯 Frage: Kann Gradientenabstieg die gleiche Lösung finden?")
print("💡 Erwartung: Klare positive Steigung (mehr Kühlenergie bei höherer Temperatur)")
print("🔮 Anwendung: Energieplanung für verschiedene Wetterbedingungen")

## 🧮 MSE-Funktion definieren

**MSE (Mean Squared Error)** = Mittlerer quadrierter Fehler

$$\text{MSE}(m,b) = \frac{1}{n} \sum_{i=1}^{n} (y_i - (mx_i + b))^2$$

**Das ist exakt die Kostenfunktion aus der Vorlesung!**

Für unsere drei Punkte (10,14), (20,19), (30,25):
- **Normalgleichungen**: Finden das Minimum durch $\frac{\partial \text{MSE}}{\partial m} = 0$ und $\frac{\partial \text{MSE}}{\partial b} = 0$
- **Gradientenabstieg**: Folgt dem Gradienten iterativ bergab zum Minimum

**Beide Methoden sollten zum gleichen Ergebnis führen!**

In [ ]:
def calculate_mse(m, b, x_data, y_data):
    """
    Berechnet den Mean Squared Error für gegebene Parameter m und b
    """
    # Vorhersagen: y_hat = mx + b
    y_predicted = m * x_data + b
    
    # Residuen: Differenz zwischen echten und vorhergesagten Werten
    residuals = y_data - y_predicted
    
    # MSE: Mittlerer quadrierter Fehler
    mse = np.mean(residuals**2)
    
    return mse

# Test mit der analytischen Lösung (Kontrolle)
mse_check = calculate_mse(m_analytical, b_analytical, x_data, y_data)
print(f"🔍 Kontrolle - MSE der analytischen Lösung: {mse_check:.6f}")
print(f"📋 Übereinstimmung: {abs(mse_check - mse_analytical) < 1e-10}")

# Test: Verwende den konfigurierten Startpunkt  
# (m_guess und b_guess wurden bereits in der Startpunkt-Konfiguration gesetzt)
mse_guess = calculate_mse(START_M, START_B, x_data, y_data)

print(f"\n🤔 Konfigurierter Startpunkt: m = {START_M}, b = {START_B}")
print(f"📊 MSE = {mse_guess:.3f}")

# Berechnung per Hand für unsere Schätzung (Demonstration)
print("\n✋ Per Hand gerechnet (zur Kontrolle):")
total_squared_error = 0
for i, (x, y) in enumerate(zip(x_data, y_data)):
    y_pred = START_M * x + START_B
    residual = y - y_pred
    squared_residual = residual**2
    total_squared_error += squared_residual
    print(f"Punkt {i+1}: ({x}°C, {y} kWh/h) → Vorhersage {y_pred:.1f} → Residuum {residual:.1f} → r² = {squared_residual:.3f}")

manual_mse = total_squared_error / len(x_data)
print(f"\n🧮 MSE per Hand: {manual_mse:.3f}")
print(f"🖥️  MSE per Code: {mse_guess:.3f}")
print("✅ Stimmen überein!")

print(f"\n💭 Die Frage:")
print(f"   • Wie kann der Computer automatisch von MSE={mse_guess:.3f} zu MSE={mse_analytical:.6f} kommen?")
print(f"   • Antwort: Gradientenabstieg!")

In [ ]:
def calculate_gradients_correct(m, b, x_data, y_data):
    """
    Berechnet die Gradienten der MSE-Funktion korrekt für m und b.
    
    Aus der Vorlesung: MSE(m,b) = 1/n * Σ(y_i - (mx_i + b))²
    
    Partielle Ableitungen:
    ∂MSE/∂m = -2/n * Σ[x_i * (y_i - (mx_i + b))]
    ∂MSE/∂b = -2/n * Σ[y_i - (mx_i + b)]
    """
    n = len(x_data)
    y_pred = m * x_data + b
    residuals = y_data - y_pred
    
    # Gradient nach m: -2/n * Summe[x_i * (y_i - (mx_i + b))]
    grad_m = -2 * np.sum(x_data * residuals) / n
    # Gradient nach b: -2/n * Summe[y_i - (mx_i + b)]
    grad_b = -2 * np.sum(residuals) / n
    
    return grad_m, grad_b

# Test der Gradienten am optimalen Punkt (sollten ~0 sein)
grad_m_opt, grad_b_opt = calculate_gradients_correct(m_analytical, b_analytical, x_data, y_data)
print(f"🎯 Gradienten am Optimum (sollten ≈ 0 sein):")
print(f"   ∂MSE/∂m = {grad_m_opt:.8f}")
print(f"   ∂MSE/∂b = {grad_b_opt:.8f}")
print(f"✅ Praktisch null - Normalgleichungen haben das Minimum gefunden!")

# Test der Gradienten am schlechten Startpunkt
grad_m_start, grad_b_start = calculate_gradients_correct(START_M, START_B, x_data, y_data)
print(f"\n🤔 Gradienten am Startpunkt m={START_M}, b={START_B}:")
print(f"   ∂MSE/∂m = {grad_m_start:.4f}")
print(f"   ∂MSE/∂b = {grad_b_start:.4f}")
print(f"💡 Diese zeigen die Richtung des steilsten Anstiegs!")
print(f"🏔️  Für Abstieg gehen wir in die entgegengesetzte Richtung!")

## 🗺️ Die MSE-Landschaft erstellen

Jetzt kommt das Spannende! Wir berechnen MSE für viele verschiedene Kombinationen von **m** (Steigung) und **b** (Achsenabschnitt) und visualisieren das als "Gebirge".

## 🌄 Interaktive 3D-MSE-Landschaft

**Jetzt wird's spannend!** Wir erstellen eine **interaktive 3D-Visualisierung** der MSE-Landschaft, die du mit der Maus drehen und zoomen kannst!

In [ ]:
# Alternative für interaktive 3D-Plots - BOKEH ist besser für Jupyter!
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Bokeh als beste Alternative für echte Interaktivität in Jupyter
try:
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.models import HoverTool, ColorBar, LinearColorMapper
    from bokeh.palettes import Plasma256
    from bokeh.layouts import row, column
    from bokeh.models import ColumnDataSource
    import bokeh.io
    
    # Bokeh für Jupyter Notebook aktivieren
    output_notebook()
    BOKEH_AVAILABLE = True
    print(">>> Bokeh gefunden und für Jupyter konfiguriert!")
    print("*** Bokeh funktioniert deutlich besser als Plotly in Notebooks!")
    
except ImportError:
    BOKEH_AVAILABLE = False
    print(">>> Bokeh nicht verfügbar - verwende Matplotlib")
    print("!!! Installiere Bokeh für echte Interaktivität: pip install bokeh")
except Exception as e:
    BOKEH_AVAILABLE = False
    print(f"!!! Bokeh-Konfigurationsproblem: {e}")
    print(">>> Verwende Matplotlib als Fallback")

print(">>> 3D-Visualisierung wird erstellt...")

# Parameter-Bereiche für 3D-Plot definieren (größerer Bereich für Startpunkt)
m_range_3d = np.linspace(-0.1, 1.2, 60)   # Erweitert um Startpunkt m=0.3 zu erfassen
b_range_3d = np.linspace(0, 25, 60)       # Erweitert um Startpunkt b=16.0 zu erfassen

# Gitter erstellen
M_3d, B_3d = np.meshgrid(m_range_3d, b_range_3d)

# MSE für alle Kombinationen berechnen
MSE_3d = np.zeros_like(M_3d)

print(f"\n>>> Berechne MSE-Landschaft (höhere Auflösung: {len(m_range_3d)}×{len(b_range_3d)})...")
for i in range(M_3d.shape[0]):
    for j in range(M_3d.shape[1]):
        MSE_3d[i, j] = calculate_mse(M_3d[i, j], B_3d[i, j], x_data, y_data)

print("*** MSE-Landschaft berechnet!")

In [ ]:
# 🌋 INTERAKTIVE MSE-LANDSCHAFT (Bokeh ist die beste Lösung für Jupyter!)
# =========================================================================

if BOKEH_AVAILABLE:
    print(">>> Erstelle interaktive Bokeh-Visualisierung...")
    
    try:
        # Daten für Bokeh vorbereiten (2D Heatmap + Konturen)
        # Bokeh macht zwar keine echten 3D-Plots, aber sehr schöne interaktive 2D-Plots
        
        # 1. INTERAKTIVE HEATMAP der MSE-Landschaft
        # Flatten der Daten für Bokeh
        m_flat = M_3d.flatten()
        b_flat = B_3d.flatten()  
        mse_flat = MSE_3d.flatten()
        
        # ColumnDataSource für Bokeh
        source = ColumnDataSource(data=dict(
            m=m_flat,
            b=b_flat,
            mse=mse_flat
        ))
        
        # Color mapper für MSE-Werte
        color_mapper = LinearColorMapper(palette=Plasma256, 
                                       low=mse_flat.min(), 
                                       high=mse_flat.max())
        
        # Hauptplot erstellen
        p = figure(width=600, height=500,
                  title="Interaktive MSE-Energielandschaft",
                  x_axis_label="Steigung m",
                  y_axis_label="Y-Achsenabschnitt b",
                  toolbar_location="above")
        
        # Heatmap/Scatter plot
        scatter = p.scatter('m', 'b', size=8, source=source,
                          color={'field': 'mse', 'transform': color_mapper},
                          alpha=0.8)
        
        # Optimum markieren
        p.scatter([m_analytical], [b_analytical], size=20, color='red', 
                 marker='star', line_color='white', line_width=2, 
                 legend_label=f'* Optimum ({m_analytical:.3f}, {b_analytical:.3f})')
        
        # Startpunkt markieren  
        p.scatter([START_M], [START_B], size=15, color='lime', marker='circle',
                 line_color='black', line_width=2,
                 legend_label=f'> Start ({START_M}, {START_B})')
        
        # Hover-Tool für Interaktivität
        hover = HoverTool(tooltips=[
            ("Steigung m", "@m{0.000}"),
            ("Y-Achsenabschnitt b", "@b{0.000}"), 
            ("MSE", "@mse{0.000000}")
        ])
        p.add_tools(hover)
        
        # Color bar hinzufügen
        color_bar = ColorBar(color_mapper=color_mapper, width=8, location=(0,0))
        p.add_layout(color_bar, 'right')
        
        # 2. ZUSÄTZLICHER QUERSCHNITT-PLOT
        mse_slice = [calculate_mse(m, b_analytical, x_data, y_data) for m in m_range_3d]
        
        p2 = figure(width=600, height=300,
                   title=f"MSE-Querschnitt bei b={b_analytical:.2f}",
                   x_axis_label="Steigung m",
                   y_axis_label="MSE")
        
        p2.line(m_range_3d, mse_slice, line_width=3, color='purple', 
               legend_label='MSE(m)')
        p2.line([m_analytical, m_analytical], [0, max(mse_slice)], 
               line_width=2, color='red', line_dash='dashed',
               legend_label=f'Optimum m={m_analytical:.3f}')
        p2.line([START_M, START_M], [0, max(mse_slice)],
               line_width=2, color='green', line_dash='dashed', 
               legend_label=f'Start m={START_M}')
        
        # Beide Plots anzeigen
        layout = column(p, p2)
        show(layout)
        
        print("*** INTERAKTIVE BOKEH-VISUALISIERUNG AKTIV!")
        print("=" * 60)
        print(">>> INTERAKTIVE FEATURES:")
        print("   • Hover über Punkte → MSE-Werte anzeigen")
        print("   • Pan-Tool → Verschieben der Ansicht")
        print("   • Zoom-Tool → Hinein-/Herauszoomen")
        print("   • Reset-Tool → Zurück zur ursprünglichen Ansicht")
        print()
        print(">>> Was du siehst:")
        print("   • Roter Stern = Optimum (Tiefster MSE-Wert)")
        print("   • Grüner Kreis = Startschätzung")
        print("   • Farbverlauf = MSE-Landschaft (dunkel = niedrig, hell = hoch)")
        print("   • Unterer Plot = Querschnitt durch die Landschaft")
        print("   • >>> Gradientenabstieg würde vom grünen zum roten Punkt laufen!")
        
    except Exception as bokeh_error:
        print(f"!!! Bokeh-Fehler: {bokeh_error}")
        print(">>> Verwende Matplotlib als Fallback...")
        BOKEH_AVAILABLE = False

# HAUPTVISUALISIERUNG: 3D MSE-LANDSCHAFT
print("*** Erstelle große 3D-MSE-Landschaft mit Startpunkt...")

# Große 3D-Visualisierung mit Fokus auf 3D-Plot
fig = plt.figure(figsize=(20, 8))

# 3D Surface Plot (größer und zentraler)
ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(M_3d, B_3d, MSE_3d, 
                      cmap='viridis', alpha=0.8, linewidth=0, antialiased=True,
                      rstride=1, cstride=1)

# Optimum und Startpunkt markieren
start_mse = calculate_mse(START_M, START_B, x_data, y_data)
ax1.scatter([m_analytical], [b_analytical], [mse_analytical], 
           color='red', s=300, alpha=1.0, label='* Optimum', edgecolors='white', linewidth=2)
ax1.scatter([START_M], [START_B], [start_mse], 
           color='lime', s=250, alpha=1.0, label='> Start', edgecolors='black', linewidth=2)

# Schönere Achsenbeschriftung
ax1.set_xlabel('Steigung m', fontsize=12, fontweight='bold')
ax1.set_ylabel('Y-Achsenabschnitt b', fontsize=12, fontweight='bold')
ax1.set_zlabel('MSE', fontsize=12, fontweight='bold')
ax1.set_title('3D MSE-Landschaft (Erweitert)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)

# Colorbar für 3D-Plot
fig.colorbar(surf, ax=ax1, shrink=0.5, aspect=30)

# Bessere Ansicht einstellen (um 90° gedreht für bessere Perspektive)
ax1.view_init(elev=20, azim=135)

# Konturkarte mit mehr Details
ax2 = fig.add_subplot(122)
contour = ax2.contourf(M_3d, B_3d, MSE_3d, levels=25, cmap='viridis', alpha=0.8)
contour_lines = ax2.contour(M_3d, B_3d, MSE_3d, levels=15, colors='white', alpha=0.6, linewidths=0.8)
ax2.clabel(contour_lines, inline=True, fontsize=8, fmt='%.2f')

# Punkte markieren
ax2.plot(m_analytical, b_analytical, 'r*', markersize=20, label='* Optimum', 
         markeredgecolor='white', markeredgewidth=2)
ax2.plot(START_M, START_B, 'o', color='lime', markersize=15, label='> Start',
         markeredgecolor='black', markeredgewidth=2)

ax2.set_xlabel('Steigung m', fontsize=12, fontweight='bold')
ax2.set_ylabel('Y-Achsenabschnitt b', fontsize=12, fontweight='bold')
ax2.set_title('MSE-Konturen (von oben)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

# Achsenbereiche erweitern für bessere Übersicht
ax2.set_xlim(m_range_3d.min() - 0.05, m_range_3d.max() + 0.05)
ax2.set_ylim(b_range_3d.min() - 1, b_range_3d.max() + 1)

# Colorbar für Konturkarte
plt.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()

print("*** GROßE 3D MSE-LANDSCHAFT ERSTELLT!")
print("=" * 60)
print(">>> Was du siehst:")
print("   • Links: 3D-Oberfläche der MSE-Landschaft (drehbar im interaktiven Modus)")
print("   • Rechts: Konturkarte von oben (Höhenlinien mit MSE-Werten)")
print()
print("!!! INTERPRETATION:")
print(f"   • Roter Stern = Optimum bei m={m_analytical:.3f}, b={b_analytical:.3f}")
print(f"   • Grüner Kreis = Startpunkt bei m={START_M}, b={START_B}")
print(f"   • MSE-Bereich: {MSE_3d.min():.3f} bis {MSE_3d.max():.1f}")
print("   • Tiefe Täler = Niedrige MSE (besser)")
print("   • Hohe Berge = Hohe MSE (schlechter)")
print()
print(">>> Gradientenabstieg würde vom grünen Punkt bergab zum roten Stern laufen!")

print(f"\n!!! EMPFEHLUNG für echte Interaktivität:")
print(f"   pip install bokeh")  
print(f"   Bokeh funktioniert viel besser als Plotly in Jupyter Notebooks!")
print(f"   Mit Bokeh bekommst du echte Mouse-Over-Effekte und Zoom-Funktionen!")

In [ ]:
# 🏔️ ALTERNATIVE: NOCH GRÖßERE 3D-LANDSCHAFT
# =============================================

# Für eine noch bessere Übersicht eine extra große Visualisierung
print("*** Erstelle maximale 3D-MSE-Landschaft...")

fig = plt.figure(figsize=(16, 12))

# Großer 3D-Plot
ax = fig.add_subplot(111, projection='3d')

# Surface mit noch mehr Details
surf = ax.plot_surface(M_3d, B_3d, MSE_3d, 
                      cmap='viridis', alpha=0.9, linewidth=0, antialiased=True,
                      rstride=1, cstride=1, shade=True)

# Optimum und Startpunkt extra groß markieren
start_mse = calculate_mse(START_M, START_B, x_data, y_data)
ax.scatter([m_analytical], [b_analytical], [mse_analytical], 
          color='red', s=400, alpha=1.0, label='* Optimum', 
          edgecolors='white', linewidth=3, depthshade=False)
ax.scatter([START_M], [START_B], [start_mse], 
          color='lime', s=350, alpha=1.0, label='> Start', 
          edgecolors='black', linewidth=3, depthshade=False)

# Schöne Achsenbeschriftung
ax.set_xlabel('Steigung m', fontsize=14, fontweight='bold', labelpad=10)
ax.set_ylabel('Y-Achsenabschnitt b', fontsize=14, fontweight='bold', labelpad=10)
ax.set_zlabel('MSE', fontsize=14, fontweight='bold', labelpad=10)
ax.set_title('3D MSE-Landschaft - Komplette Übersicht', fontsize=16, fontweight='bold', pad=20)

# Legende größer
ax.legend(fontsize=12, loc='upper right')

# Colorbar
cbar = fig.colorbar(surf, ax=ax, shrink=0.6, aspect=40, pad=0.1)
cbar.set_label('MSE-Wert', fontsize=12, fontweight='bold')

# Optimale 3D-Ansicht (um 90° gedreht für bessere Perspektive)
ax.view_init(elev=25, azim=135)

# Gitter für bessere Orientierung
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("*** MAXIMALE 3D-VISUALISIERUNG FERTIG!")
print("=" * 60)
print(">>> Parameter-Bereiche:")
print(f"   • Steigung m: {m_range_3d.min():.1f} bis {m_range_3d.max():.1f}")
print(f"   • Y-Achsenabschnitt b: {b_range_3d.min():.1f} bis {b_range_3d.max():.1f}")
print(f"   • MSE-Bereich: {MSE_3d.min():.3f} bis {MSE_3d.max():.1f}")
print()
print(">>> Wichtige Punkte:")
print(f"   • Optimum: m={m_analytical:.3f}, b={b_analytical:.3f}, MSE={mse_analytical:.6f}")
print(f"   • Start: m={START_M}, b={START_B}, MSE={start_mse:.3f}")
print(f"   • Verbesserung: {start_mse/mse_analytical:.1f}x niedriger MSE beim Optimum!")
print()
print("!!! Diese Landschaft zeigt perfekt, wie Gradientenabstieg funktioniert:")
print("   1. Start auf dem hohen 'Berg' (grüner Punkt)")
print("   2. Folge dem steilsten Abstieg (Gradientenrichtung)")
print("   3. Erreiche das tiefste Tal (rotes Optimum)")

In [ ]:
# 🔄 Zurück zu statischen Plots für den Rest des Notebooks
%matplotlib inline

print(">>> Matplotlib auf statische Plots zurückgesetzt für weitere Visualisierungen")

## 🚶‍♂️ Gradientenabstieg Schritt für Schritt

Jetzt simulieren wir, wie der Computer den Weg bergab findet! Wir starten von unserer Schätzung aus und folgen dem steilsten Abstieg.

⚠️ **Wichtig**: Die **Lernrate α** ist kritisch! Zu groß → Divergenz, zu klein → sehr langsam.

In [ ]:
# Für eine stabile Demonstration fügen wir diskret einige zusätzliche Punkte hinzu
x_demo = np.array([8, 10, 12, 15, 18, 20, 22, 25, 27, 30, 32, 35])
y_demo = np.array([12.5, 14, 15.2, 16.8, 18.1, 19, 20.5, 22.3, 23.8, 25, 26.1, 28.2])

# WICHTIG: Analytische Lösung für die erweiterten Punkte berechnen
n_demo = len(x_demo)
sum_x_demo = np.sum(x_demo)
sum_y_demo = np.sum(y_demo)
sum_xy_demo = np.sum(x_demo * y_demo)
sum_x2_demo = np.sum(x_demo**2)

m_demo_optimal = (n_demo * sum_xy_demo - sum_x_demo * sum_y_demo) / (n_demo * sum_x2_demo - sum_x_demo**2)
b_demo_optimal = (sum_y_demo - m_demo_optimal * sum_x_demo) / n_demo
mse_demo_optimal = calculate_mse(m_demo_optimal, b_demo_optimal, x_demo, y_demo)

print("📊 ANALYTISCHE LÖSUNG für die erweiterten Punkte:")
print(f"   Anzahl Punkte: {n_demo}")
print(f"   Optimale Steigung: m = {m_demo_optimal:.6f}")
print(f"   Optimaler Y-Achsenabschnitt: b = {b_demo_optimal:.6f}")
print(f"   Optimaler MSE: {mse_demo_optimal:.8f}")

print(f"\n🔍 Vergleich Original vs. Erweitert:")
print(f"   Original (3 Punkte): m = {m_analytical:.6f}, b = {b_analytical:.6f}")
print(f"   Erweitert ({n_demo} Punkte): m = {m_demo_optimal:.6f}, b = {b_demo_optimal:.6f}")

def gradient_descent_simple(x_data, y_data, start_m=0.3, start_b=16.0, learning_rate=0.01, max_iterations=200):
    """Einfacher Gradientenabstieg für Demonstrationszwecke"""
    m, b = start_m, start_b
    history = {'m': [m], 'b': [b], 'mse': []}
    
    for i in range(max_iterations):
        # MSE berechnen
        mse = calculate_mse(m, b, x_data, y_data)
        history['mse'].append(mse)
        
        # Gradienten berechnen
        grad_m, grad_b = calculate_gradients_correct(m, b, x_data, y_data)
        
        # Parameter aktualisieren
        m = m - learning_rate * grad_m
        b = b - learning_rate * grad_b
        
        # Historie speichern
        history['m'].append(m)
        history['b'].append(b)
        
        # Fortschritt anzeigen (alle 1000 Iterationen)
        if (i + 1) % 1000 == 0:
            print(f"Iteration {i+1:5d}: m = {m:.6f}, b = {b:.6f}, MSE = {mse:.8f}")
    
    # Finale MSE
    final_mse = calculate_mse(m, b, x_data, y_data)
    history['mse'].append(final_mse)
    
    return history

print(f"\n🚀 Gradientenabstieg ist bereit!")
print(f"📊 Ziel: m = {m_demo_optimal:.6f}, b = {b_demo_optimal:.6f}")

In [ ]:
# INFO: Startpunkt-Konfiguration wurde bereits früher im Notebook definiert
# Die Variablen START_M und START_B sind bereits gesetzt und werden hier nur referenziert

print("=== STARTPUNKT FÜR GRADIENTENABSTIEG ===")
print(f"Verwendeter Startpunkt: m = {START_M}, b = {START_B}")

# Erweiterte Validierung für den Gradientenabstieg
if 'x_demo' in locals():
    start_mse_value = calculate_mse(START_M, START_B, x_demo, y_demo)
    print(f"MSE am Startpunkt: {start_mse_value:.4f}")
    
    if 'mse_demo_optimal' in locals():
        print(f"Ziel-MSE (optimal): {mse_demo_optimal:.6f}")
        print(f"Verbesserungspotential: {start_mse_value/mse_demo_optimal:.1f}x")

# Validierung des Startpunkts
if START_M < -1.0 or START_M > 2.0:
    print("!!! WARNUNG: Steigung außerhalb des sinnvollen Bereichs (-1.0 bis 2.0)")
if START_B < -10.0 or START_B > 40.0:
    print("!!! WARNUNG: Y-Achsenabschnitt außerhalb des sinnvollen Bereichs (-10 bis 40)")

print(f"\n>>> Bereit für Gradientenabstieg von ({START_M}, {START_B}) zum Optimum!")

In [ ]:
# 🧪 EXPERIMENTIER-BEREICH: Ändere die Lernrate hier!
# ===============================================================

LEARNING_RATE = 0.0015  # ⚠️ OPTIMAL GEFUNDEN! 
# Funktioniert: 0.0015 mit 20000 Iterationen
# Andere Werte zum Testen: 0.001, 0.002, 0.005

print("🎯 Starte Gradientenabstieg...")
print(f"📍 Startpunkt: m = {START_M}, b = {START_B}")
print(f"🎯 ZIEL (korrekt für erweiterte Daten): m = {m_demo_optimal:.6f}, b = {b_demo_optimal:.6f}")
print(f"🔧 Lernrate: {LEARNING_RATE}")
print("-" * 70)

# Gradientenabstieg ausführen
history = gradient_descent_simple(x_demo, y_demo, start_m=START_M, start_b=START_B, 
                                learning_rate=LEARNING_RATE, max_iterations=20000)

# Finale Ergebnisse
final_m = history['m'][-1]
final_b = history['b'][-1]
final_mse = history['mse'][-1]

print("-" * 70)
print("🎉 FERTIG!")
print(f"📊 Ergebnis: m = {final_m:.6f}, b = {final_b:.6f}")
print(f"📈 Finale MSE: {final_mse:.8f}")

# Vergleich mit analytischer Lösung (KORREKT für erweiterte Daten)
print(f"✅ Optimal MSE: {mse_demo_optimal:.8f}")
print(f"📊 MSE-Differenz: {abs(final_mse - mse_demo_optimal):.8f}")
print(f"📊 Parameter-Differenz: Δm = {abs(final_m - m_demo_optimal):.6f}, Δb = {abs(final_b - b_demo_optimal):.6f}")

if abs(final_mse - mse_demo_optimal) < 0.001:
    print("🎉 PERFEKT! Gradientenabstieg hat das Optimum erreicht!")
elif abs(final_mse - mse_demo_optimal) < 0.01:
    print("✅ SEHR GUT! Gradientenabstieg ist sehr nah am Optimum!")
elif abs(final_mse - mse_demo_optimal) < 0.1:
    print("👍 GUT! Gradientenabstieg konvergiert zum Optimum!")
else:
    print("⚠️ ACHTUNG! Möglicherweise divergiert oder Lernrate zu groß/klein!")
    
# Konvergenz-Check
if len(history['mse']) > 10:
    recent_change = abs(history['mse'][-1] - history['mse'][-10])
    if recent_change < 1e-6:
        print("✅ Konvergiert stabil")
    elif recent_change > 1000:
        print("🚨 DIVERGENZ! Lernrate zu groß!")
    else:
        print("📈 Noch am konvergieren...")

print(f"\n💡 TIPP: Experimentiere mit verschiedenen Lernraten!")
print(f"   • Zu groß (z.B. 0.1): Divergenz oder Oszillation")
print(f"   • Zu klein (z.B. 0.0001): Sehr langsame Konvergenz")
print(f"   • Optimal (z.B. 0.01): Schnelle, stabile Konvergenz")

In [ ]:
# Visualisierung der Konvergenz
plt.figure(figsize=(15, 5))

# Plot 1: MSE-Verlauf
plt.subplot(1, 3, 1)
plt.plot(history['mse'], 'purple', linewidth=2, marker='o', markersize=3)
plt.axhline(y=mse_demo_optimal, color='red', linestyle='--', alpha=0.7, 
           label=f'Optimum: {mse_demo_optimal:.6f}')
plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.title('MSE-Minimierung')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Log-Skala für bessere Sichtbarkeit

# Plot 2: Parameter-Entwicklung m
plt.subplot(1, 3, 2)
plt.plot(history['m'], 'blue', linewidth=2, marker='o', markersize=3, label='Steigung m')
plt.axhline(y=m_demo_optimal, color='red', linestyle='--', alpha=0.7, 
           label=f'Optimum: {m_demo_optimal:.4f}')
plt.xlabel('Iteration')
plt.ylabel('Steigung m')
plt.title('Konvergenz der Steigung')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Parameter-Entwicklung b
plt.subplot(1, 3, 3)
plt.plot(history['b'], 'green', linewidth=2, marker='o', markersize=3, label='Y-Achsenabschnitt b')
plt.axhline(y=b_demo_optimal, color='red', linestyle='--', alpha=0.7, 
           label=f'Optimum: {b_demo_optimal:.4f}')
plt.xlabel('Iteration')
plt.ylabel('Y-Achsenabschnitt b')
plt.title('Konvergenz des Y-Achsenabschnitts')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Die Plots zeigen die schrittweise Verbesserung der Parameter!")
print("💡 MSE ist auf logarithmischer Skala für bessere Sichtbarkeit")

In [ ]:
# Finale Vergleichstabelle und Visualisierung
comparison_data = {
    'Methode': ['Startschätzung', 'Gradientenabstieg', 'Analytisch (optimal)'],
    'Steigung m': [START_M, final_m, m_demo_optimal],
    'Achsenabschnitt b': [START_B, final_b, b_demo_optimal],
    'MSE': [calculate_mse(START_M, START_B, x_demo, y_demo), final_mse, mse_demo_optimal]
}

df = pd.DataFrame(comparison_data)
print("ERGEBNISVERGLEICH (für erweiterte Daten):")
print("=" * 70)
print(df.to_string(index=False, float_format='%.6f'))

# Visualisierung der drei Geraden
plt.figure(figsize=(12, 8))

# Datenpunkte
plt.scatter(x_demo, y_demo, color='red', s=100, zorder=5, edgecolors='black', 
           linewidth=1, label='Demonstrationsdaten', alpha=0.8)

# Geraden
x_line = np.linspace(5, 40, 100)
y_start = START_M * x_line + START_B
y_gradient = final_m * x_line + final_b
y_optimal = m_demo_optimal * x_line + b_demo_optimal

plt.plot(x_line, y_start, 'b:', linewidth=3, alpha=0.7,
         label=f'Start: y = {START_M}x + {START_B} (MSE={comparison_data["MSE"][0]:.4f})')
plt.plot(x_line, y_gradient, 'g--', linewidth=3, 
         label=f'Gradientenabstieg: y = {final_m:.4f}x + {final_b:.4f} (MSE={final_mse:.4f})')
plt.plot(x_line, y_optimal, 'r-', linewidth=3, alpha=0.8,
         label=f'Optimal: y = {m_demo_optimal:.4f}x + {b_demo_optimal:.4f} (MSE={mse_demo_optimal:.4f})')

plt.xlabel('Außentemperatur (°C)', fontweight='bold')
plt.ylabel('Energieverbrauch (kWh/h)', fontweight='bold')
plt.title('Gradientenabstieg vs. Analytische Lösung', fontweight='bold', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.xlim(5, 40)
plt.ylim(10, 30)

plt.tight_layout()
plt.show()

# Erfolgsauswertung
if abs(final_mse - mse_demo_optimal) < 0.001:
    print("\n🎉 PERFEKTER ERFOLG! Gradientenabstieg hat die optimale Lösung gefunden!")
    print("💡 Der Computer kann automatisch die beste Gerade durch die Punkte finden!")
elif abs(final_mse - mse_demo_optimal) < 0.01:
    print("\n✅ SEHR GUTER ERFOLG! Gradientenabstieg ist sehr nah an der optimalen Lösung!")
    print("💡 Mit mehr Iterationen oder angepasster Lernrate wäre perfekte Konvergenz möglich!")
else:
    print("\n⚠️ Bitte Lernrate anpassen! Aktuell noch nicht optimal konvergiert.")
    print("💡 Versuchen Sie verschiedene Lernraten in der vorherigen Zelle!")